### SEVER DEFINITIONS


In [13]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv
import copy
import os
from rtls_slave import aoa_main, pixel_calculate


# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    global aoa
    global aoabefore
    lev.append(horiz)
    print("Plane Angle: " + str(horiz))
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_main()
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)
    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./aoa_results/{slave.replace(':','_')}_aoa_data_before_calibration.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        

        for i in range(minnumber):
            for j in range(minnumber):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j])

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
        
        f.close()

    
    return jsonify({'state': 'success'})

### RUNNING THE SEVER

In [14]:
app.run(host='0.0.0.0',port = 5000, debug=False) 

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


Plane Angle: 124
Master : <RTLSNode(CC26x2 Master, started daemon 21524)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 21524)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -94, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -91, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : F8:8A:5E:2D:89:EA
Connected to : F8:8A:5E:2D:89:EA with connection handle 0
AOA Params Set for Connection Handle: 0
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected


10.19.190.195 - - [18/Mar/2023 11:49:56] "GET /send/124 HTTP/1.1" 200 -


Done
{'F8:8A:5E:2D:89:EA': [-3, -13, -20, -19, -20, -15, -18, -17, -2, -2, 3, 0, 0, -18, -19, -14, -16, -10, -8, -5, -6, -16, -18, -23]}
Plane Angle: 124
Master : <RTLSNode(CC26x2 Master, started daemon 9752)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 9752)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -95, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -59, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -96, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : F8:8A:5E:2D:89:EA
Connected to : F8:8A:5E:2D:89:EA with connection handle 0
AOA Params Set for Connection Handle: 0
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected


...

Before Turning：{'80:6F:B0:EE:9C:67': [-15, -12, -10, -9, -10, -10, -10, -11, -12, -14, -15, -13, -13, -14, -14, -13, -14, -11, -11, -10, -12, -10, -10, -12, -13], 'F8:8A:5E:2D:89:EA': [-14, -6, -9, -8, -11, -18, -19, -13, -13, -9, -6, -5, -9, -8, -15, -17, -17, -18, -16, -17]}
After Turning： {'80:6F:B0:EE:9C:67': [-15, -12, -10, -9, -10, -10, -10, -11, -12, -14, -15, -13, -13, -14, -14, -13, -14, -11, -11, -10, -12, -10, -10, -12, -13], 'F8:8A:5E:2D:89:EA': [-14, -6, -9, -8, -11, -18, -19, -13, -13, -9, -6, -5, -9, -8, -15, -17, -17, -18, -16, -17]}
Rotation Angle： [124, 124, 124, 138, 138]
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected


10.19.190.195 - - [18/Mar/2023 11:54:04] "GET /send/138 HTTP/1.1" 200 -


Done
{'80:6F:B0:EE:9C:67': [-10, -9, -10, -9, -10, -11, -11, -12, -9, -11, -9, -10, -8, -9, -11, -9, -12, -11, -12, -12, -10, -12, -10, -9], 'F8:8A:5E:2D:89:EA': [-16, -9, -14, -13, -15, -1, 4, 5, 3, -3, -12, -10, -10, -10, -8, -9, -1, -4, -2, -4, -3, -4]}


f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:246: RuntimeWarning: divide by zero encountered in double_scalars
  elvation1 = np.arccos((cosAOAangle2-coslevelangle*cosAOAangle1)/sinlevelangle)
f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:246: RuntimeWarning: invalid value encountered in arccos
  elvation1 = np.arccos((cosAOAangle2-coslevelangle*cosAOAangle1)/sinlevelangle)
f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:246: RuntimeWarning: invalid value encountered in double_scalars
  elvation1 = np.arccos((cosAOAangle2-coslevelangle*cosAOAangle1)/sinlevelangle)
10.19.190.195 - - [18/Mar/2023 11:54:10] "GET /calculate/ HTTP/1.1" 200 -


Before Turning：{'80:6F:B0:EE:9C:67': [-15, -12, -10, -9, -10, -10, -10, -11, -12, -14, -15, -13, -13, -14, -14, -13, -14, -11, -11, -10, -12, -10, -10, -12, -13], 'F8:8A:5E:2D:89:EA': [-14, -6, -9, -8, -11, -18, -19, -13, -13, -9, -6, -5, -9, -8, -15, -17, -17, -18, -16, -17]}
After Turning： {'80:6F:B0:EE:9C:67': [-10, -9, -10, -9, -10, -11, -11, -12, -9, -11, -9, -10, -8, -9, -11, -9, -12, -11, -12, -12, -10, -12, -10, -9], 'F8:8A:5E:2D:89:EA': [-16, -9, -14, -13, -15, -1, 4, 5, 3, -3, -12, -10, -10, -10, -8, -9, -1, -4, -2, -4, -3, -4]}
Rotation Angle： [124, 124, 124, 138, 138]
